In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('../data/processed/final_merged_events.csv', low_memory=False)

In [5]:
combined_data.count()

NameError: name 'combined_data' is not defined

In [30]:
combined_data.head(30491)

,id,item_id,dept_id,cat_id,store_id,state_id,date,units_sold,wm_yr_wk,sell_price,sales,event_name,event_count
0,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-29,3,11101,2.00,6.00,NoEvent,0.0
1,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-30,0,11101,2.00,0.00,NoEvent,0.0
2,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-01-31,0,11101,2.00,0.00,NoEvent,0.0
3,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-02-01,1,11101,2.00,2.00,NoEvent,0.0
4,FOODS_1_001_CA_1_evaluation,FOODS_1_001,FOODS_1,FOODS,CA_1,CA,2011-02-02,4,11101,2.00,8.00,NoEvent,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30486,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-20,3,11416,0.98,2.94,NoEvent,0.0
30487,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-21,5,11416,0.98,4.90,NoEvent,0.0
30488,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-22,3,11416,0.98,2.94,NoEvent,0.0
30489,FOODS_1_021_CA_1_evaluation,FOODS_1_021,FOODS_1,FOODS,CA_1,CA,2014-05-23,5,11416,0.98,4.90,NoEvent,0.0


In [13]:
count_rows = combined_data[(combined_data['item_id'] == 'FOODS_1_021') & (combined_data['store_id'] == 'CA_1')].shape[0]
print(count_rows)

1541


In [15]:
unique_item_count = len(target_item_ids)
print("Number of unique item_ids:", unique_item_count)


Number of unique item_ids: 3049


In [3]:
import pandas as pd

# Assuming 'date' is in a string format, convert it to datetime
data['date'] = pd.to_datetime(data['date'])

# Now you can use .dt accessor to extract date components
data['day_of_week'] = data['date'].dt.dayofweek
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year


In [ ]:
#combined_data['rolling_7d_mean'] = combined_data.groupby(['item_id', 'store_id'])['units_sold'].rolling(window=7).mean().reset_index(level=0, drop=True)
#combined_data['rolling_30d_sum'] = combined_data.groupby(['item_id', 'store_id'])['units_sold'].rolling(window=30).sum().reset_index(level=0, drop=True)


In [ ]:
#combined_data['units_sold_lag_7'] = combined_data.groupby(['item_id', 'store_id'])['units_sold'].shift(7)


In [ ]:
import os  # Import the os module
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from category_encoders import TargetEncoder
import joblib  # Import joblib

# List of store names
store_names = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

# Define the groups based on your criteria
group1 = ['CA_1', 'CA_2', 'CA_3', 'CA_4']
group2 = ['TX_1', 'TX_2', 'TX_3']
group3 = ['WI_1', 'WI_2', 'WI_3']

# Initialize a dictionary to store the trained models and encoders for each group
models = {}

# Iterate over each group and filter the data accordingly
for group_idx, group in enumerate([group1, group2, group3]):
    # Filter the data for the current group
    group_data = data[data['store_id'].isin(group)]
    
    # Define features and target variable
    date_features = ['day_of_week', 'month', 'year']
    categorical_features = ['store_id', 'item_id']
    target = 'sales'
    
    # Split the data into training, validation, and test sets
    train_data, test_data = train_test_split(group_data, test_size=0.2, random_state=42)
    train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

    # Extract features and target variables for training, validation, and test sets
    X_train, y_train = train_data[date_features + categorical_features], train_data[target]
    X_val, y_val = val_data[date_features + categorical_features], val_data[target]
    X_test, y_test = test_data[date_features + categorical_features], test_data[target]

    # Initialize target encoders for categorical features
    target_encoders = {}
    for feature in categorical_features:
        target_encoder = TargetEncoder()
        X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
        X_val[feature] = target_encoder.transform(X_val[feature])
        X_test[feature] = target_encoder.transform(X_test[feature])
        target_encoders[feature] = target_encoder

    # Initialize and train the Random Forest model on the encoded feature set
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Make predictions on the validation set
    val_predictions = model.predict(X_val)

    # Evaluate the model on the validation set using Mean Absolute Error (MAE)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'Group {group_idx + 1} - Validation MAE with Target Encoding: {val_mae}')

    # Store the trained model and encoders in the dictionary for this group
    model_and_encoders = {
        'model': model,
        'encoders': target_encoders
    }
    
    # Define the models directory
    models_dir = "../models/Randomforest"
    os.makedirs(models_dir, exist_ok=True)  # Create the directory if it doesn't exist
    
    # Define the model file path for saving
    model_file_path = os.path.join(models_dir, f'model_group_{group_idx + 1}.joblib')
    
    # Save the model to the specified file path using joblib
    joblib.dump(model_and_encoders, model_file_path)

# You can access the models and encoders for each group using the 'models' dictionary


/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[feature] = target_encoder.fit_transform(X_train[feature], y_train)
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val[feature] = target_encoder.transform(X_val[feature])
/var/folders/dm/ly88h28j28b8nml3dg1my5bc0000gn/T/ipykernel_3643/2399735645.py:45: SettingWithCopyWarning: 
A value is tryin

In [1]:
import pandas as pd
import joblib

# Load the trained model for the specific group (e.g., Group 1)
model_group = 1
model_file_path = f"../models/model_group_{model_group}.joblib"
loaded_model = joblib.load(model_file_path)

# Define the input data for prediction
input_data = pd.DataFrame({
    'day_of_week': [1],  # Replace with the day of the week for your date
    'month': [6],        # Replace with the month for your date
    'year': [2016],      # Replace with the year for your date
    'store_id': ['CA_1'],  # Replace with the desired store_id
    'item_id': ['FOODS_1_001']  # Replace with the desired item_id
})

# Perform target encoding on categorical features using the loaded encoders
for feature, encoder in loaded_model['encoders'].items():
    input_data[feature] = encoder.transform(input_data[feature])

# Make predictions using the loaded model
predicted_sales = loaded_model['model'].predict(input_data)

# Print the predicted sales
print(f"Predicted Sales: {predicted_sales[0]}")


TypeError: list indices must be integers or slices, not str